<a href="https://colab.research.google.com/github/herysedra/ady_cov/blob/master/scr/modely_SIR/covmdg_no_change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import pickle



confirmed_cases_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
confirmed_cases = pd.read_csv(confirmed_cases_url, sep=',')
deaths_url =  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
deaths = pd.read_csv(deaths_url, sep=',')
path_to_save = '/content/sample_data/covmdg1/'
path_data = '/content/sample_data/covmdg1/'


In [0]:
def delay_cases(new_I_t, len_new_I_t, len_new_cases_obs , delay, delay_arr):


    delay_mat = make_delay_matrix(n_rows=len_new_I_t, 
                                  n_columns=len_new_cases_obs, initial_delay=delay_arr)
    inferred_cases = interpolate(new_I_t, delay, delay_mat)
    return inferred_cases 

def make_delay_matrix(n_rows, n_columns, initial_delay=0):
    """
    Has in each entry the delay between the input with size n_rows and the output
    with size n_columns
    """
    size = max(n_rows, n_columns)
    mat = np.zeros((size, size))
    for i in range(size):
        diagonal = np.ones(size-i)*(initial_delay + i)
        mat += np.diag(diagonal, i)
    for i in range(1, size):
        diagonal = np.ones(size-i)*(initial_delay - i)
        mat += np.diag(diagonal, -i)
    return mat[:n_rows, :n_columns]
    

def interpolate(array, delay, delay_matrix):
    interp_matrix = tt.maximum(1-tt.abs_(delay_matrix - delay), 0)
    interpolation = tt.dot(array,interp_matrix)
    return interpolation


In [0]:
import pymc3 as pm
import theano.tensor as tt
import theano
import datetime
import time

"""
Mahasedra comments:
libraries and modules : pymc3 (on Bayesian statistical modeling),  
theano (for manipulating and evaluating mathematical expressions, especially 
matrix-valued ones)
"""

date_data_begin = datetime.date(2020,4,1)
date_data_end = datetime.date(2020,4,15)
num_days_to_predict = 28

"""
Mahasedra:
Above are the period of interest from which the data will be retrieved.
"""

diff_data_sim = 16 # should be significantly larger than the expected delay, in 
                   # order to always fit the same number of data points.

"""
Because of the reporting delay, in order to analyze the reported new cases 
between date_data_begin and date_data_end, we have to look at simulated/inferred
values of new cases at time starting before t - delay.
"""


date_begin_sim = date_data_begin - datetime.timedelta(days = diff_data_sim)

"""
Mahasedra: (corrected)
diff_data_sim: number of days which separate the date_data_begin (21 march 2020) 
and the date_begin_sim (14 february 2020: the simulation starts )
"""
format_date = lambda date_py: '{}/{}/{}'.format(date_py.month, date_py.day,
                                                 str(date_py.year)[2:4])
date_formatted_begin = format_date(date_data_begin)
date_formatted_end = format_date(date_data_end)

"""
Mahasedra & Joely: format date
"""

cases_obs =  np.array(
    confirmed_cases.loc[confirmed_cases["Country/Region"] == "Madagascar", 
                        date_formatted_begin:date_formatted_end])[0]
#cases_obs = np.concatenate([np.nan*np.ones(diff_data_sim), cases_obs])

"""
Mahasedra: (corrected)
cases_obs : the number of infected cases in tana from the 
date_data_begin (1 march 2020) to the date_begin_sim (14 february 2020).
"""

print('Cases of ({}): {} and '
      'day before that: {}'.format(date_data_end.isoformat(), *cases_obs[:-3:-1]))

"""
Mahasedra & Joely: output about the new cases at the begin day of 
the forecasting and at the day before.
"""

num_days = (date_data_end - date_begin_sim).days
# 
date_today = date_data_end + datetime.timedelta(days=1)

print(cases_obs)

Cases of (2020-04-15): 110 and day before that: 108
[ 57  59  70  70  72  82  88  93  93  93 102 106 106 108 110]


In [0]:

# ------------------------------------------------------------------------------ #
# model setup and training
# ------------------------------------------------------------------------------ #
np.random.seed(0)

def SIR_model(λ, μ, S_begin, I_begin, N):
    new_I_0 = tt.zeros_like(I_begin)
    def next_day(λ, S_t, I_t, _):
        new_I_t = λ/N*I_t*S_t
        S_t = S_t - new_I_t
        I_t = I_t + new_I_t - μ * I_t
        return S_t, I_t, new_I_t
    outputs , _  = theano.scan(fn=next_day, sequences=[λ], 
                               outputs_info=[S_begin, I_begin, new_I_0])
    S_all, I_all, new_I_all = outputs
    return S_all, I_all, new_I_all

    """
    Mahasedra:
    SIR_model(): provides all values of S, I and new infected cases given
    by the SIR model with the initial conditions S_begin, I_begin, N.
    To be checked
    """



with pm.Model() as model:
    # true cases at begin of loaded data but we do not know the real number
    I_begin = pm.HalfCauchy('I_begin', beta=100)

    # fraction of people that are newly infected each day
    λ = pm.Lognormal("λ", mu=np.log(0.4), sigma=0.5)

    # fraction of people that recover each day, recovery rate mu
    μ = pm.Lognormal('μ', mu=np.log(1/8), sigma=0.2)

    # delay in days between contracting the disease and being recorded
    delay = pm.Lognormal("delay", mu=np.log(8), sigma=0.2)

    # prior of the error of observed cases
    σ_obs = pm.HalfCauchy("σ_obs", beta=10)

    N_tana = 2e6

    """
    Mahasedra: I suppose that only part of the population can be really susceptible
    to be infected.
    """

    # -------------------------------------------------------------------------- #
    # training the model with loaded data
    # -------------------------------------------------------------------------- #

    S_begin = N_tana - I_begin
    S_past, I_past, new_I_past = SIR_model(λ=λ * tt.ones(num_days-1), μ=μ, 
                                               S_begin=S_begin, I_begin=I_begin,
                                               N=N_tana)
    new_cases_obs = np.diff(cases_obs)
    new_cases_inferred = delay_cases(new_I_past, len_new_I_t=num_days - 1, 
                                     len_new_cases_obs=len(new_cases_obs), 
                                     delay=delay, delay_arr=diff_data_sim)

    # Approximates Poisson
    # calculate the likelihood of the model:
    # observed cases are distributed following studentT around the model
    pm.StudentT(
        "obs",
        nu=4,
        mu=new_cases_inferred,
        sigma=(new_cases_inferred)**0.5 * σ_obs,
        observed=new_cases_obs)  
    
    S_past = pm.Deterministic('S_past', S_past)
    I_past = pm.Deterministic('I_past', I_past)
    new_I_past = pm.Deterministic('new_I_past', new_I_past)
    new_cases_past = pm.Deterministic('new_cases_past', new_cases_inferred)

    # -------------------------------------------------------------------------- #
    # prediction, start with no changes in policy
    # -------------------------------------------------------------------------- #

    S_begin = S_past[-1]
    I_begin = I_past[-1]
    forecast_no_change = SIR_model(λ=λ*tt.ones(num_days_to_predict), μ=μ, 
                        S_begin=S_begin, I_begin=I_begin, N=N_tana)
    S_no_change, I_no_change, new_I_no_change = forecast_no_change

    #saves the variables for later retrieval
    pm.Deterministic('S_no_change', S_no_change)
    pm.Deterministic('I_no_change', I_no_change)
    pm.Deterministic('new_I_no_change', new_I_no_change)

    """
    PyMC3 allows you to freely do algebra with RVs in all kinds of ways (...).
    While these transformations work seamlessly, their results are not 
    stored automatically. Thus, if you want to keep track of a transformed 
    variable, you have to use pm.Deterministic.
    """

    new_cases_inferred = delay_cases(tt.concatenate([new_I_past[-diff_data_sim:], new_I_no_change]), 
                                     len_new_I_t=diff_data_sim + num_days_to_predict, 
                                     len_new_cases_obs=num_days_to_predict, 
                                     delay=delay, delay_arr=diff_data_sim)
    pm.Deterministic('new_cases_no_change', new_cases_inferred)

    time_beg = time.time()
    trace = pm.sample(draws=3000, tune=800, chains=2)
    print("Model run in {:.2f} s".format(time.time() - time_beg))


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
Sequential sampling (2 chains in 1 job)
NUTS: [σ_obs, delay, μ, λ, I_begin]
100%|██████████| 3800/3800 [01:07<00:00, 56.10it/s]
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


Model run in 206.95 s


In [0]:
C_nc = np.array(new_I_no_change)
C_nc

array(Subtensor{int64::}.0, dtype=object)

In [0]:
legends_lang = {
    "english": [
        # bottom left
        "Confirmed cases",
        [
            "No social distancing",
            "Mild social distancing",
            "Strong social distancing",
            "Strong social distancing since 5 days ago",
        ],
        "Days since intervention",
        "Total confirmed cases in Tana",
        "Start of intervention",
        # bottom right
        'Confirmed cases',['Strong social distancing:',
                           '  starting at day 0', 
                           '  starting at day 5',
                           "  starting at day -5"],
        'Days since intervention',
        'Total confirmed cases in Madagascar',
        'Start of intervention',
        "Spreading\nrate",
        ['Strong social distancing',
         'Strong social distancing\n  with long transient', 
         '  with immediate change'],
    ],
}

obs_cases_labels = ['new_cases_no_change', 'new_cases_soc_dist', 
                    'new_cases_isol', 'new_cases_isol_later','new_cases_earlier', 
                    'new_cases_long_trans','new_cases_immedi']

# label the variables related to the new_cases_inferred

cases_obs_to_plot = np.array(confirmed_cases.loc[confirmed_cases['Country/Region'] == 'Madagascar', 
                                                 date_formatted_begin:date_formatted_end])[0]
cases_obs_to_plot_future = np.array(confirmed_cases.loc[confirmed_cases['Country/Region'] == 'Madagascar', date_formatted_end:])[0]

print(cases_obs_to_plot)
print(cases_obs_to_plot_future)

[ 57  59  70  70  72  82  88  93  93  93 102 106 106 108 110]
[110 111 117]
